In [398]:
import pandas as pd
import numpy as np

모든 데이터 통합하기
-------------------------------------------------------------------------------
> 1. 환경기상/gs/유동인구/sns로 저장된 데이터를 하나의 파일로 통합
> 2. PCA를 통해 다중공선성이 높은 pm25와 pm10을 차원축소
> 3. 수치높은데이터(People, Count)는 log를 취함

데이터 불러오기
-------------------------------------------------------------------------------
> 환경기상/gs/유동인구/sns를 불러옵니다.

In [399]:
df_sns=pd.read_csv('./pre_sns_count.csv', encoding='utf-8')
df_pop=pd.read_csv('./pre_flow.csv', encoding='utf-8')
df_air=pd.read_csv('./pre_air_out.csv', encoding='utf-8')
df_gs=pd.read_csv('./pre_gs.csv', encoding='utf-8')


/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [400]:
df_gs = df_gs.rename(columns={"Years": "YEAR", "Days": "DAY", "Months" : "MONTH", "ADMD_CD":"serial"})
df_gs = df_gs.drop(['BOR_CD','OPER_DT', 'week', 'weekday', 'weekend','AMT_IND'], axis = 1)
df_gs.head()

,serial,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P,YEAR,MONTH,DAY
0,11110515,27.9,30.0,33.5,1.6,5.4,0.000000,1.3,0.4,2018,4,1
1,11110530,23.7,25.8,35.0,4.6,9.3,0.500000,0.2,0.9,2018,4,1
2,11110540,25.4,24.7,32.4,6.5,9.0,0.685234,1.3,0.7,2018,4,1
3,11110550,41.6,21.8,28.3,2.7,3.7,0.600000,1.2,0.2,2018,4,1
4,11110560,31.1,26.7,26.2,1.5,13.5,0.685234,0.7,0.2,2018,4,1


In [401]:
df_pop = df_pop.rename(columns={"STD_Y": "YEAR", "STD_D": "DAY", "STD_M" : "MONTH", "HDONG_NM": "serial"})
df_pop.head()

,serial,YEAR,MONTH,DAY,Man,Woman
0,11110515,2018,4,1,20859,15614
1,11110530,2018,4,1,71961,59010
2,11110540,2018,4,1,13988,11317
3,11110550,2018,4,1,40905,28830
4,11110560,2018,4,1,78602,59849


In [402]:
df_air = df_air.drop(['hours'], axis = 1)
df_air = df_air.rename(columns={"Year": "YEAR", "Days": "DAY", "Month" : "MONTH"})
df_air = df_air.groupby(['YEAR','MONTH','DAY','serial']).agg(np.mean)

df_air = pd.DataFrame(df_air)
df_air.head()

pm10       pm25
YEAR MONTH DAY serial                        
2018 4     1   11110515  66.041667  31.375000
               11110560  76.291667  36.041667
               11110615  60.500000  29.333333
               11110630  60.458333  28.541667
               11110640  60.416667  28.416667

In [403]:
print(df_air.shape)
df_air.head()

(8912, 2)


pm10       pm25
YEAR MONTH DAY serial                        
2018 4     1   11110515  66.041667  31.375000
               11110560  76.291667  36.041667
               11110615  60.500000  29.333333
               11110630  60.458333  28.541667
               11110640  60.416667  28.416667

데이터 통합오기
-------------------------------------------------------------------------------
> df_all로 환경기상/gs/유동인구/sns를 merge합니다.

In [404]:
df_all = pd.merge(left=df_gs, right=df_air, how='left',on =['YEAR','MONTH','DAY','serial'])

df_all = pd.merge(left=df_all, right=df_pop, how='left',on =['YEAR','MONTH','DAY','serial'])

df_all = pd.merge(left=df_all, right=df_sns, how='left',on =['YEAR','MONTH','DAY'])


In [405]:
print(df_all.shape)
df_all.head()

(11812, 17)


,serial,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P,YEAR,MONTH,DAY,pm10,pm25,Man,Woman,Count
0,11110515,27.9,30.0,33.5,1.6,5.4,0.000000,1.3,0.4,2018,4,1,66.041667,31.375000,20859,15614,5020
1,11110530,23.7,25.8,35.0,4.6,9.3,0.500000,0.2,0.9,2018,4,1,NaN,NaN,71961,59010,5020
2,11110540,25.4,24.7,32.4,6.5,9.0,0.685234,1.3,0.7,2018,4,1,NaN,NaN,13988,11317,5020
3,11110550,41.6,21.8,28.3,2.7,3.7,0.600000,1.2,0.2,2018,4,1,NaN,NaN,40905,28830,5020
4,11110560,31.1,26.7,26.2,1.5,13.5,0.685234,0.7,0.2,2018,4,1,76.291667,36.041667,78602,59849,5020


In [406]:
df_all.isnull().sum()

serial          0
LCLS_10_P       0
LCLS_20_P       0
LCLS_30_P       0
LCLS_40_P       0
LCLS_50_P       0
LCLS_60_P       0
LCLS_70_P       0
LCLS_80_P       0
YEAR            0
MONTH           0
DAY             0
pm10         9329
pm25         9329
Man             0
Woman           0
Count           0
dtype: int64

In [407]:
df_all = df_all.dropna()

In [408]:
print(df_all.shape)
df_all.head()

(2483, 17)


,serial,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P,YEAR,MONTH,DAY,pm10,pm25,Man,Woman,Count
0,11110515,27.9,30.0,33.5,1.6,5.4,0.000000,1.3,0.4,2018,4,1,66.041667,31.375000,20859,15614,5020
4,11110560,31.1,26.7,26.2,1.5,13.5,0.685234,0.7,0.2,2018,4,1,76.291667,36.041667,78602,59849,5020
8,11110615,27.5,21.0,38.7,2.4,7.6,0.600000,1.7,0.5,2018,4,1,60.500000,29.333333,198378,156244,5020
9,11110640,22.3,21.8,37.5,4.6,11.0,0.400000,2.0,0.5,2018,4,1,60.416667,28.416667,34126,33440,5020
10,11110650,31.7,20.7,34.2,2.6,8.1,0.400000,2.2,0.1,2018,4,1,60.291667,28.458333,47641,41422,5020


In [409]:
# #연월일로 되어있는 것을 Datetime으로 바꿈 (시간은 제외)
df_all['Datetime']=df_all['YEAR'].astype('str')+'-'+df_all['MONTH'].astype('str')+'-'+df_all['DAY'].astype('str')
df_all['Datetime']=pd.to_datetime(df_all['Datetime'], format='%Y-%m-%d')

In [410]:
print(df_all.shape)
df_all.head()

(2483, 18)


,serial,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P,YEAR,MONTH,DAY,pm10,pm25,Man,Woman,Count,Datetime
0,11110515,27.9,30.0,33.5,1.6,5.4,0.000000,1.3,0.4,2018,4,1,66.041667,31.375000,20859,15614,5020,2018-04-01
4,11110560,31.1,26.7,26.2,1.5,13.5,0.685234,0.7,0.2,2018,4,1,76.291667,36.041667,78602,59849,5020,2018-04-01
8,11110615,27.5,21.0,38.7,2.4,7.6,0.600000,1.7,0.5,2018,4,1,60.500000,29.333333,198378,156244,5020,2018-04-01
9,11110640,22.3,21.8,37.5,4.6,11.0,0.400000,2.0,0.5,2018,4,1,60.416667,28.416667,34126,33440,5020,2018-04-01
10,11110650,31.7,20.7,34.2,2.6,8.1,0.400000,2.2,0.1,2018,4,1,60.291667,28.458333,47641,41422,5020,2018-04-01


다중공선성 방지위해 PCA(Feature Extraction)
-------------------------------------------------------------------------------
> 환경기상데이터의 pm25,pm10을 PCA를 이용하여 차원을 축소합니다.

In [411]:
pm25 = []
pm25 = df_all['pm25']

pm10 = []
pm10 = df_all['pm10']
print(type(pm10))

<class 'pandas.core.series.Series'>


In [412]:
pm_pca = pd.concat([pm25,pm10], axis = 1)

In [413]:
df_all.drop(['pm25','pm10'], axis=1, inplace=True)

In [414]:
pm_pca.head()

,pm25,pm10
0,31.375000,66.041667
4,36.041667,76.291667
8,29.333333,60.500000
9,28.416667,60.416667
10,28.458333,60.291667


In [415]:
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
pca_f = pca.fit_transform(pm_pca)
pca_df = pd.DataFrame(data = pca_f
             , columns = ['pm'])

In [416]:
pca_df.isnull().sum()

pm    0
dtype: int64

In [417]:
print(pca_df.shape)
pca_df.head()

(2483, 1)


,pm
0,12.702166
1,23.915082
2,6.879481
3,6.350967
4,6.263287


In [418]:
df_all.isnull().sum()

serial       0
LCLS_10_P    0
LCLS_20_P    0
LCLS_30_P    0
LCLS_40_P    0
LCLS_50_P    0
LCLS_60_P    0
LCLS_70_P    0
LCLS_80_P    0
YEAR         0
MONTH        0
DAY          0
Man          0
Woman        0
Count        0
Datetime     0
dtype: int64

In [419]:
df_all = df_all.reset_index(drop=True)
print(df_all.shape)
df_all.head()

(2483, 16)


,serial,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P,YEAR,MONTH,DAY,Man,Woman,Count,Datetime
0,11110515,27.9,30.0,33.5,1.6,5.4,0.000000,1.3,0.4,2018,4,1,20859,15614,5020,2018-04-01
1,11110560,31.1,26.7,26.2,1.5,13.5,0.685234,0.7,0.2,2018,4,1,78602,59849,5020,2018-04-01
2,11110615,27.5,21.0,38.7,2.4,7.6,0.600000,1.7,0.5,2018,4,1,198378,156244,5020,2018-04-01
3,11110640,22.3,21.8,37.5,4.6,11.0,0.400000,2.0,0.5,2018,4,1,34126,33440,5020,2018-04-01
4,11110650,31.7,20.7,34.2,2.6,8.1,0.400000,2.2,0.1,2018,4,1,47641,41422,5020,2018-04-01


In [420]:
pca_df = pca_df.reset_index(drop=True)
print(pca_df.shape)
pca_df.head()

(2483, 1)


,pm
0,12.702166
1,23.915082
2,6.879481
3,6.350967
4,6.263287


In [421]:
df_all[['pm']] = pca_df[['pm']]

In [422]:
df_all.isnull().sum()

serial       0
LCLS_10_P    0
LCLS_20_P    0
LCLS_30_P    0
LCLS_40_P    0
LCLS_50_P    0
LCLS_60_P    0
LCLS_70_P    0
LCLS_80_P    0
YEAR         0
MONTH        0
DAY          0
Man          0
Woman        0
Count        0
Datetime     0
pm           0
dtype: int64

In [423]:
print(df_all.shape)
df_all

(2483, 17)


,serial,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P,YEAR,MONTH,DAY,Man,Woman,Count,Datetime,pm
0,11110515,27.9,30.0,33.5,1.6,5.4,0.000000,1.300000,0.400000,2018,4,1,20859,15614,5020,2018-04-01,12.702166
1,11110560,31.1,26.7,26.2,1.5,13.5,0.685234,0.700000,0.200000,2018,4,1,78602,59849,5020,2018-04-01,23.915082
2,11110615,27.5,21.0,38.7,2.4,7.6,0.600000,1.700000,0.500000,2018,4,1,198378,156244,5020,2018-04-01,6.879481
3,11110640,22.3,21.8,37.5,4.6,11.0,0.400000,2.000000,0.500000,2018,4,1,34126,33440,5020,2018-04-01,6.350967
4,11110650,31.7,20.7,34.2,2.6,8.1,0.400000,2.200000,0.100000,2018,4,1,47641,41422,5020,2018-04-01,6.263287
5,11110670,33.6,20.9,36.1,1.1,3.6,2.100000,2.600000,0.100000,2018,4,1,27828,20125,5020,2018-04-01,-1.854008
6,11110515,27.0,24.6,40.5,3.7,3.1,0.100000,0.300000,0.700000,2018,4,2,30379,19649,6316,2018-04-02,8.109150
7,11110615,27.3,19.4,42.6,2.8,5.9,0.300000,1.500000,0.100000,2018,4,2,409733,272719,6316,2018-04-02,2.938190
8,11110630,24.6,16.7,48.0,2.6,6.1,0.300000,1.100000,0.500000,2018,4,2,111276,83449,6316,2018-04-02,2.281128
9,11110640,27.6,20.7,37.5,5.0,6.6,0.800000,1.700000,0.100000,2018,4,2,54452,50547,6316,2018-04-02,4.616606


In [424]:
df_all = df_all.set_index(["Datetime"])

In [425]:
df_all

,serial,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P,YEAR,MONTH,DAY,Man,Woman,Count,pm
Datetime,,,,,,,,,,,,,,,,
2018-04-01,11110515,27.9,30.0,33.5,1.6,5.4,0.000000,1.300000,0.400000,2018,4,1,20859,15614,5020,12.702166
2018-04-01,11110560,31.1,26.7,26.2,1.5,13.5,0.685234,0.700000,0.200000,2018,4,1,78602,59849,5020,23.915082
2018-04-01,11110615,27.5,21.0,38.7,2.4,7.6,0.600000,1.700000,0.500000,2018,4,1,198378,156244,5020,6.879481
2018-04-01,11110640,22.3,21.8,37.5,4.6,11.0,0.400000,2.000000,0.500000,2018,4,1,34126,33440,5020,6.350967
2018-04-01,11110650,31.7,20.7,34.2,2.6,8.1,0.400000,2.200000,0.100000,2018,4,1,47641,41422,5020,6.263287
2018-04-01,11110670,33.6,20.9,36.1,1.1,3.6,2.100000,2.600000,0.100000,2018,4,1,27828,20125,5020,-1.854008
2018-04-02,11110515,27.0,24.6,40.5,3.7,3.1,0.100000,0.300000,0.700000,2018,4,2,30379,19649,6316,8.109150
2018-04-02,11110615,27.3,19.4,42.6,2.8,5.9,0.300000,1.500000,0.100000,2018,4,2,409733,272719,6316,2.938190
2018-04-02,11110630,24.6,16.7,48.0,2.6,6.1,0.300000,1.100000,0.500000,2018,4,2,111276,83449,6316,2.281128


In [426]:
# 다중공선성 막기 위해 상관관계가 높은 변수 처리 (남,여 유동인구수)
df_all['Pop'] = df_all['Man'] + df_all['Woman']
df_all.drop(['Man', 'Woman'], axis=1, inplace=True)


In [427]:
df_all.head()

,serial,LCLS_10_P,LCLS_20_P,LCLS_30_P,LCLS_40_P,LCLS_50_P,LCLS_60_P,LCLS_70_P,LCLS_80_P,YEAR,MONTH,DAY,Count,pm,Pop
Datetime,,,,,,,,,,,,,,,
2018-04-01,11110515,27.9,30.0,33.5,1.6,5.4,0.000000,1.3,0.4,2018,4,1,5020,12.702166,36473
2018-04-01,11110560,31.1,26.7,26.2,1.5,13.5,0.685234,0.7,0.2,2018,4,1,5020,23.915082,138451
2018-04-01,11110615,27.5,21.0,38.7,2.4,7.6,0.600000,1.7,0.5,2018,4,1,5020,6.879481,354622
2018-04-01,11110640,22.3,21.8,37.5,4.6,11.0,0.400000,2.0,0.5,2018,4,1,5020,6.350967,67566
2018-04-01,11110650,31.7,20.7,34.2,2.6,8.1,0.400000,2.2,0.1,2018,4,1,5020,6.263287,89063


In [428]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2483 entries, 2018-04-01 to 2019-03-31
Data columns (total 15 columns):
serial       2483 non-null object
LCLS_10_P    2483 non-null float64
LCLS_20_P    2483 non-null float64
LCLS_30_P    2483 non-null float64
LCLS_40_P    2483 non-null float64
LCLS_50_P    2483 non-null float64
LCLS_60_P    2483 non-null float64
LCLS_70_P    2483 non-null float64
LCLS_80_P    2483 non-null float64
YEAR         2483 non-null int64
MONTH        2483 non-null int64
DAY          2483 non-null int64
Count        2483 non-null int64
pm           2483 non-null float64
Pop          2483 non-null int64
dtypes: float64(9), int64(5), object(1)
memory usage: 390.4+ KB


In [429]:
#큰수는 데이터에 로그값을 취합니다.
df_all['Pop'] =np.log(df_all['Pop'])

In [430]:
df_all.to_csv('df_all.csv')